<center>
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

# **Arize Agent Mastry Course: Agent Architectures**

In this lab, we’ll explore agent architectures by implementing two common frameworks. Understanding different architectures is key to identifying which approach best fits your workflow and use case.

We’ll use the same setup as before, then leverage features of the Agno framework to demonstrate both the Orchestrator–Worker architecture and a Parallelization architecture. Finally, we’ll examine the traces of each framework within Arize to better understand their behavior and performance.

# Set Up

In [1]:
!pip install -qqqqqq arize-otel agno openai openinference-instrumentation-agno openinference-instrumentation-openai httpx

In [3]:
import os
from getpass import getpass
from google.colab import userdata

os.environ["ARIZE_SPACE_ID"] = userdata.get("ARIZE_SPACE_ID") or getpass("🔑 Enter your Arize Space ID: ")

os.environ["ARIZE_API_KEY"] = userdata.get("ARIZE_API_KEY") or getpass("🔑 Enter your Arize API Key: ")

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API Key: ")

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY") or getpass("🔑 Enter your Tavily API Key: ")

In [4]:
from arize.otel import register, Transport
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

model_id = "travel-agent-demo"
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name=model_id,
    set_global_tracer_provider=True,
    log_to_console=True,
    endpoint="https://otlp.ca-central-1a.arize.com/v1/traces",
    transport=Transport.HTTP
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: travel-agent-demo
|  Span Processor: Multiple Span Processors
|  Collector Endpoint: Multiple Span Exporters
|  Transport: Multiple Span Exporters
|  Transport Headers: Multiple Span Exporters
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Define Tools

In [5]:
# --- Helper functions for tools ---
import httpx

def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None


def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]


In [6]:
from agno.tools import tool

@tool
def essential_info(destination: str) -> str:
    q = f"{destination} travel essentials weather best time top attractions etiquette"
    s = _search_api(q)
    if s:
        return f"{destination} essentials: {_compact(s)}"
    return f"{destination} is a popular travel destination. Expect local culture, cuisine, and landmarks worth exploring."

@tool
def budget_basics(destination: str, duration: str) -> str:
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."

@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    q = f"{destination} authentic local experiences {interests}"
    s = _search_api(q)
    if s:
        return f"{destination} {interests}: {_compact(s)}"
    return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."


# Agent Architecture 1: Orchestrator-Worker Framework

In the Orchestrator–Worker framework, we will structure our system using multiple specialized sub-agents (one for each of our 3 tools).

Each sub-agent focuses on a specific capability, such as getting essential information, estimating budgets, or suggesting local experiences.

A centralized orchestrator agent coordinates these sub-agents by delegating tasks to the appropriate one and then synthesizing their outputs into a cohesive final response. This approach mirrors how complex workflows can be broken down into smaller, focused tasks that work together seamlessly.

![Diagram](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-orchestrator-worker-diagram.png)

In [7]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# --- Define Subagents ---
destination_agent = Agent(
    name="DestinationInfo",
    model=OpenAIChat(id="gpt-4o", temperature=0.2),
    description="Get basic travel info (weather, best time, attractions, etiquette).",
    instructions=["Provide concise, reliable travel info for a destination using the essential_info tool."],
    tools=[essential_info],
)

budget_agent = Agent(
    name="Budget",
    model=OpenAIChat(id="gpt-4o"),
    description="Summarize travel cost.",
    instructions=["Give clear travel budget summaries with hotel, meal, and transport cost ranges; give multiple options with prices and locations."],
    tools = [budget_basics],
    markdown=True,
)

local_activity_agent = Agent(
    name="ActivitySuggester",
    model=OpenAIChat(id="gpt-4o", temperature=0.8),
    description="Suggest authentic local experiences.",
    instructions=[
        "Group local activities by category: cultural, food, outdoors.",
        "Include both popular and hidden-gem recommendations."
    ],
    tools=[local_flavor],
    markdown=True,
)


In [8]:
from agno.team import Team

travel_team = Team(
    name="Orchestrator-TripPlanner",
    members=[destination_agent, budget_agent, local_activity_agent],
    model=OpenAIChat(id="gpt-4o"),
    instructions=[
        "You are a friendly and knowledgeable travel planner. "
        "Combine coordinate agents to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ],
    show_members_responses=True,
    markdown=True,
)



In [9]:
# --- Example usage ---
destination = "Tokyo"
duration = "5 days"
interests = "food, culture"

query = f"""
Plan a {duration} trip to {destination}.
Focus on {interests}.
Include essential info, budget breakdown, and local experiences.
"""
travel_team.print_response(
  query,
  stream=True
)

Output()

{
    "name": "ChatCompletion",
    "context": {
        "trace_id": "0x045d8478c308f4ad7b8b285b014afe6b",
        "span_id": "0x93241bc2aaa8bead",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x3599afe6a1a7c488",
    "start_time": "2026-01-03T15:32:06.344748Z",
    "end_time": "2026-01-03T15:32:08.732993Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "llm.provider": "openai",
        "llm.system": "openai",
        "input.value": "{\"messages\": [{\"role\": \"developer\", \"content\": \"You are the leader of a team and sub-teams of AI Agents.\\nYour task is to coordinate the team to complete the user's request.\\n\\nHere are the members in your team:\\n<team_members>\\n - Agent 1:\\n   - ID: destinationinfo\\n   - Name: DestinationInfo\\n - Agent 2:\\n   - ID: budget\\n   - Name: Budget\\n - Agent 3:\\n   - ID: activitysuggester\\n   - Name: ActivitySuggester\\n</team_members>\\n\\n<how_to_respond>\\n- Your ro

Tracing this agent reveals the Orchestrator–Worker workflow in action. We can observe how tasks are explicitly delegated to individual sub-agents and how their outputs are combined to produce the final response.

![Trace](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-orchestrator-worker-trace.png)

# Agent Architecture 2: Parallelization Framework

In the Parallelization framework, we run all sub-agents concurrently instead of sequentially. Each sub-agent works independently on its assigned task — for example, retrieving essential information, estimating budgets, or finding local experiences — while the main agent waits to gather their results. Once all sub-agents complete their work, the agent synthesizes their outputs into a unified response.

This approach offers a significant latency advantage, as parallel execution reduces overall response time without compromising the quality or completeness of the final answer.

![Diagram](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-parallelization.png)

In [ ]:
import asyncio
from agno.agent import Agent
from agno.models.openai import OpenAIChat


In [ ]:
# --- Define Subagents ---
destination_agent = Agent(
    name="DestinationInfo",
    model=OpenAIChat(id="gpt-4o", temperature=0.2),
    tools=[essential_info],
    instructions=["Provide concise, reliable travel info for a destination using the essential_info tool."],
)

budget_agent = Agent(
    name="Budget",
    model=OpenAIChat(id="gpt-4o"),
    tools=[budget_basics],
    instructions=["Give clear travel budget summaries with hotel, meal, and transport cost ranges; give multiple options with prices and locations."],
)

local_activity_agent = Agent(
    name="ActivitySuggester",
    model=OpenAIChat(id="gpt-4o", temperature=0.8),
    tools=[local_flavor],
    instructions=[
        "Group local activities by category: cultural, food, outdoors.",
        "Include both popular and hidden-gem recommendations."
    ],
)

synthesizer = Agent(
    name="Synthesizer",
    model=OpenAIChat(id="gpt-4o", temperature=0.3),
    instructions=[
        "Combine partial responses into a clear, well-structured final answer."
    ],
)


In [ ]:
from opentelemetry import trace
from openinference.semconv.trace import SpanAttributes

tracer = trace.get_tracer(__name__)

# --- Run subagents concurrently and synthesize ---
async def plan_trip(destination: str, duration: str, interests: str):
    with tracer.start_as_current_span("ParallelizationAgent") as span:
      span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, "agent")
      span.set_attribute("destination", destination)
      span.set_attribute("duration", duration)
      span.set_attribute("interests", interests)

      # Run all three subagents concurrently
      dest_task = destination_agent.arun(destination)
      budget_task = budget_agent.arun(duration)
      local_task = local_activity_agent.arun(interests)

      dest_info, budget_info, activities = await asyncio.gather(dest_task, budget_task, local_task)

      # Combine results via one final LLM call
      final_prompt = f"""
      Combine the following into a cohesive, well-structured travel plan for {destination}.
      Keep it friendly, natural, and under 1000 words.

      [Destination Info]
      {dest_info}

      [Budget Summary]
      {budget_info}

      [Local Activities]
      {activities}
      """

      final_plan = await synthesizer.arun(final_prompt)
      return final_plan

In [ ]:
# --- Example usage ---
destination = "Tokyo"
duration = "5 days"
interests = "food, culture"

final = await plan_trip(destination, duration, interests)

![Traces](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-parallel-agent.png)